# Build large lookup table

In [1]:
import flashalgorithm as fc
import numpy as np
import dill as pickle
import itertools as itool
import pdb

In [2]:
comp_list = ('water', 'methane', 'ethane', 'propane')
phase_list = ('aqueous', 'vapor', 'lhc', 's1', 's2')
P = 80  # bar
T = 273.15 + 12 # Kelvin

In [3]:
flash_full = fc.FlashController(components=comp_list,
                                phases=phase_list)

In [4]:
water_fracs = [0]
hc_fracs = np.linspace(0, 1, 50)
c1_frac, c2_frac, c3_frac = np.meshgrid(hc_fracs, hc_fracs, hc_fracs)

In [5]:
z_all = list()
for water in water_fracs:
    for hcs in zip(c1_frac.flatten(), c2_frac.flatten(), c3_frac.flatten()):
        if sum(hcs) > 0.0:
            mod_hcs = [x / sum(hcs) * (1.0 - water) for x in hcs]
            z = np.asarray([water] + mod_hcs)
            z_all.append(z / np.sum(z))

In [6]:
z_use = np.unique(np.asarray(z_all), axis=0)

In [7]:
def emptycomp_hash(z):
    hashed = sum([2**ii for ii, x in enumerate(z) if x == 0.0])
    return hashed

In [8]:
flash_dict = {0: flash_full}    

In [9]:
all_output = list()
out_file = 'c1toc3_80bar12C_nowater.pkl'
for ii, z in enumerate(z_use):
    if len([x for x in z if x > 0]) > 1:
        comp_hash = emptycomp_hash(z)
        new_comps, new_z = zip(*[
                    (comp, z_) for comp, z_ in zip(comp_list, z) 
                    if z_ != 0.0
                ])
        if comp_hash not in flash_dict.keys():
            flash_dict.update({comp_hash: 
                                   fc.FlashController(
                                        components=new_comps,
                                        phases=phase_list)})

        flash_use = flash_dict[comp_hash]
        new_z = np.asarray(new_z)
        try:
            output = flash_use.main_handler(
                        compobjs=flash_use.compobjs, 
                        z=new_z, 
                        T=T, 
                        P=P)
        except:
            try:
                output = flash_use.main_handler(
                            compobjs=flash_use.compobjs, 
                            z=new_z, 
                            T=T, 
                            P=P,
                            incipient_calc=True)
            except:
                try:
                    output = flash_use.main_handler(
                                compobjs=flash_use.compobjs, 
                                z=new_z, 
                                T=T, 
                                P=P,
                                initialize=True,
                                incipient_calc=True)
                except:
                    output = []
        all_output.append([ii, z, new_comps, new_z, output])
        if np.mod(ii, 100) == 0:
            print('{0:3.3f} % complete!'.format(float(ii) * 100 / len(z_use)))
            with open(out_file, 'wb') as f:
                pickle.dump(all_output, f)

0.093 % complete!
0.187 % complete!
0.280 % complete!
0.374 % complete!
0.467 % complete!
0.560 % complete!
0.654 % complete!
0.747 % complete!
0.841 % complete!
0.934 % complete!
1.027 % complete!
1.121 % complete!
1.214 % complete!


KeyboardInterrupt: 

In [10]:
ii

1300

In [11]:
z_use[0:100]

array([[ 0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.        ,  0.        ,  0.02      ,  0.98      ],
       [ 0.        ,  0.        ,  0.02040816,  0.97959184],
       [ 0.        ,  0.        ,  0.02083333,  0.97916667],
       [ 0.        ,  0.        ,  0.0212766 ,  0.9787234 ],
       [ 0.        ,  0.        ,  0.02173913,  0.97826087],
       [ 0.        ,  0.        ,  0.02222222,  0.97777778],
       [ 0.        ,  0.        ,  0.02272727,  0.97727273],
       [ 0.        ,  0.        ,  0.02325581,  0.97674419],
       [ 0.        ,  0.        ,  0.02380952,  0.97619048],
       [ 0.        ,  0.        ,  0.02439024,  0.97560976],
       [ 0.        ,  0.        ,  0.025     ,  0.975     ],
       [ 0.        ,  0.        ,  0.02564103,  0.97435897],
       [ 0.        ,  0.        ,  0.02631579,  0.97368421],
       [ 0.        ,  0.        ,  0.02702703,  0.97297297],
       [ 0.        ,  0.        ,  0.02777778,  0.97222222],
       [ 0.        ,  0.